# 3.3 Validation
Information derived from satellite observation is useless without stating it's accuracy. 
This exercise will show you how to validate results obtained on a cloud platforms. 
- Load data
- ...


In [1]:
import openeo
import xarray as xr
import rioxarray as rio 

from sentinelhub import (SHConfig, SentinelHubRequest, DataCollection, MimeType, CRS, BBox, bbox_to_dimensions, geometry)

In [2]:
config = SHConfig()
config.sh_client_id = %env SH_CLIENT_ID
config.sh_client_secret = %env SH_CLIENT_SECRET

In [4]:
conn = openeo.connect('https://jjdxlu8vwl.execute-api.eu-central-1.amazonaws.com/production')

In [5]:
conn.authenticate_basic(username=config.sh_client_id, password=config.sh_client_secret)

<Connection to 'https://jjdxlu8vwl.execute-api.eu-central-1.amazonaws.com/production/' with BasicBearerAuth>

In [ ]:
# Use this for more 
# https://github.com/openEOPlatform/sample-notebooks/blob/main/openEO%20Platfrom%20-%20Basics.ipynb
# https://github.com/Open-EO/openeo-community-examples/tree/main/python

In [ ]:
# load snow station datasets from zenodo:: https://zenodo.org/record/5109574


In [ ]:
# load snow coverage grid


In [ ]:
# data validation
